## test

In [96]:
!python doomsayer.py \
  --mode agg \
  --input tutorial_data/NMF_M_spectra.txt \
  --verbose \
  --rank 4 \
  --report

/home/jedidiah/doomsayer/doomsayer_output already exists
All output files will be located in  /home/jedidiah/doomsayer/doomsayer_output
indexing subtypes...
Writing M matrices and RMSE per sample
Running NMF model
Running NMF with rank = 4
Generating keep and drop lists
330 potential outliers found.
Rscript will run the following command:
Rscript --vanilla generate_report.r /home/jedidiah/doomsayer/doomsayer_output/config.yaml
We recommend that you use the dev version of ggplot2 with `ggplotly()`
Install it with: `devtools::install_github('hadley/ggplot2')`
We recommend that you use the dev version of ggplot2 with `ggplotly()`
Install it with: `devtools::install_github('hadley/ggplot2')`
We recommend that you use the dev version of ggplot2 with `ggplotly()`
Install it with: `devtools::install_github('hadley/ggplot2')`
`mutate_each()` is deprecated.
Use `mutate_all()`, `mutate_at()` or `mutate_if()` instead.
To map `funs` over a selection of variables, use `mutate_at()`
Total runtime: 3